Import MobileNet

In [2]:
from keras.applications.mobilenet_v2 import MobileNetV2

In [3]:
model = MobileNetV2(weights = 'imagenet')

Loading data and generating labels

In [4]:
import numpy as np
from imageio import imread

In [23]:
# preprocessing images make the matrix between -1 and 1
from keras.applications.mobilenet_v2 import preprocess_input
from skimage.transform import resize
from keras import preprocessing

data = np.empty((106, 224,224,3))
for i in range(106):
  try:
    im = imread('covid_dataset/0{}.jpeg'.format(i+1))
  except FileNotFoundError:
    continue
  except Exception:
    im = imread('covid_dataset/0{}.png'.format(i+1))
    
  im = preprocessing.image.img_to_array(im)
  im = im.reshape((im.shape[0], im.shape[1], im.shape[2]))
  im = resize(im, output_shape=(224, 224, 3))
  im = preprocess_input(im)
  data[i] = im

In [21]:
labels = np.empty(106, dtype=int)
# covid
labels[:60] = 1
# normal
labels[46:] = 0

Test if MobileNet_v2 works

In [24]:
from keras.applications.mobilenet_v2 import decode_predictions

prediction = model.predict(data)
for decoded_prediction in decode_predictions(prediction):
  for name, desc, score in decoded_prediction:
    print('- {}  ({:.2f})%'.format(desc, 100*score))


49152/35363 [=========================================] - 0s 2us/step
- tiger_shark  (5.40)%
- nematode  (5.20)%
- trilobite  (4.91)%
- hotdog  (4.84)%
- isopod  (4.65)%
- isopod  (30.76)%
- trilobite  (15.09)%
- quill  (13.41)%
- axolotl  (2.91)%
- fountain  (2.84)%
- great_white_shark  (17.87)%
- jellyfish  (3.69)%
- tiger_shark  (3.55)%
- sturgeon  (3.09)%
- cuirass  (2.40)%
- matchstick  (7.81)%
- wall_clock  (5.45)%
- screw  (4.43)%
- switch  (3.24)%
- velvet  (1.97)%
- matchstick  (7.81)%
- wall_clock  (5.45)%
- screw  (4.43)%
- switch  (3.24)%
- velvet  (1.97)%
- quilt  (7.24)%
- French_loaf  (6.55)%
- radiator  (5.05)%
- mosquito_net  (4.71)%
- seat_belt  (2.62)%
- matchstick  (7.81)%
- wall_clock  (5.45)%
- screw  (4.43)%
- switch  (3.24)%
- velvet  (1.97)%
- fountain  (37.30)%
- geyser  (9.93)%
- axolotl  (7.39)%
- great_white_shark  (2.62)%
- grey_whale  (2.04)%
- matchstick  (7.81)%
- wall_clock  (5.45)%
- screw  (4.43)%
- switch  (3.24)%
- velvet  (1.97)%
- matchstick  (7.

Modify the model

In [45]:
# generation new output layer
from keras.layers.core import Dense
from keras import Model

covid_output = Dense(2, activation='softmax')

# take all the layers of mobilenet except for the last one
# connect thouse layers to our covid_output layer
covid_output = covid_output(model.layers[-2].output)
covid_input = model.input

covid_model = Model(inputs=covid_input, outputs=covid_output)

# leave all the original pre-trained layers as they are
# freez all the layers except fro our output layer
for layer in covid_model.layers[:-1]:
  layer.trainable = False


Compiling the model

In [47]:
covid_model.compile(
  # tells our model how to map the actual output
  # into the labels the we're using to train
  loss='sparse_categorical_crossentropy',
  optimizer='adam', # algo that adjuste the model to reduce the loss
  metrics=['accuracy']
)

Training the model

In [41]:
covid_model.fit(
  x=data,
  y=labels,
  epochs=16,
  verbose=2
)

Epoch 1/16
4/4 - 3s - loss: 0.2661 - accuracy: 0.9057 - 3s/epoch - 814ms/step
Epoch 2/16
4/4 - 3s - loss: 0.2551 - accuracy: 0.9151 - 3s/epoch - 717ms/step
Epoch 3/16
4/4 - 2s - loss: 0.2368 - accuracy: 0.9340 - 2s/epoch - 596ms/step
Epoch 4/16
4/4 - 3s - loss: 0.2341 - accuracy: 0.9151 - 3s/epoch - 655ms/step
Epoch 5/16
4/4 - 4s - loss: 0.2319 - accuracy: 0.9340 - 4s/epoch - 969ms/step
Epoch 6/16
4/4 - 3s - loss: 0.2151 - accuracy: 0.9245 - 3s/epoch - 741ms/step
Epoch 7/16
4/4 - 3s - loss: 0.2118 - accuracy: 0.9340 - 3s/epoch - 738ms/step
Epoch 8/16
4/4 - 3s - loss: 0.2057 - accuracy: 0.9434 - 3s/epoch - 756ms/step
Epoch 9/16
4/4 - 3s - loss: 0.1940 - accuracy: 0.9434 - 3s/epoch - 665ms/step
Epoch 10/16
4/4 - 3s - loss: 0.1957 - accuracy: 0.9528 - 3s/epoch - 691ms/step
Epoch 11/16
4/4 - 3s - loss: 0.1877 - accuracy: 0.9623 - 3s/epoch - 743ms/step
Epoch 12/16
4/4 - 3s - loss: 0.1836 - accuracy: 0.9528 - 3s/epoch - 658ms/step
Epoch 13/16
4/4 - 3s - loss: 0.1835 - accuracy: 0.9434 - 3s/e

Generating predictions for training data

In [43]:
predictions = covid_model.predict(data)

np.argmax(predictions, axis=1)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

Generating separate training and validation sets

In [57]:
training_ds = np.empty((60, 224, 224, 3))
training_ds[:30] = data[:30]
training_ds[30:] = data[60:90]
training_labels = np.empty(60)
training_labels[:30] = 1 # 30 first pics covide
training_labels[30:] = 0 # 30 first pics normal

validation_ds = np.empty((40, 224, 224, 3))
validation_ds[:20] = data[40:60]
validation_ds[20:] = data[86:]
validation_labels = np.empty(40)
validation_labels[:20] = 1 # 30 first pics covide
validation_labels[20:] = 0 # 30 first pics normal

In [58]:
# generation new output layer
from keras.layers.core import Dense
from keras import Model

covid_output = Dense(2, activation='softmax')

# take all the layers of mobilenet except for the last one
# connect thouse layers to our covid_output layer
covid_output = covid_output(model.layers[-2].output)
covid_input = model.input

covid_model = Model(inputs=covid_input, outputs=covid_output)

# leave all the original pre-trained layers as they are
# freez all the layers except fro our output layer
for layer in covid_model.layers[:-1]:
  layer.trainable = False


In [59]:
covid_model.compile(
  # tells our model how to map the actual output
  # into the labels the we're using to train
  loss='sparse_categorical_crossentropy',
  optimizer='adam', # algo that adjuste the model to reduce the loss
  metrics=['accuracy']
)

Training with separate validation data

In [60]:
covid_model.fit(
  x=training_ds,
  y=training_labels,
  validation_data=(validation_ds, validation_labels),
  epochs=20,
  verbose=2,
)

Epoch 1/20
2/2 - 6s - loss: 1.0102 - accuracy: 0.2500 - val_loss: 0.6796 - val_accuracy: 0.5500 - 6s/epoch - 3s/step
Epoch 2/20
2/2 - 2s - loss: 0.5877 - accuracy: 0.7167 - val_loss: 0.6366 - val_accuracy: 0.6000 - 2s/epoch - 1s/step
Epoch 3/20
2/2 - 2s - loss: 0.4488 - accuracy: 0.7833 - val_loss: 0.6405 - val_accuracy: 0.7000 - 2s/epoch - 1s/step
Epoch 4/20
2/2 - 3s - loss: 0.3732 - accuracy: 0.8167 - val_loss: 0.5805 - val_accuracy: 0.7000 - 3s/epoch - 1s/step
Epoch 5/20
2/2 - 3s - loss: 0.3374 - accuracy: 0.8667 - val_loss: 0.5221 - val_accuracy: 0.7500 - 3s/epoch - 1s/step
Epoch 6/20
2/2 - 3s - loss: 0.3118 - accuracy: 0.9000 - val_loss: 0.4714 - val_accuracy: 0.8250 - 3s/epoch - 1s/step
Epoch 7/20
2/2 - 3s - loss: 0.2868 - accuracy: 0.9333 - val_loss: 0.4322 - val_accuracy: 0.8250 - 3s/epoch - 1s/step
Epoch 8/20
2/2 - 3s - loss: 0.2657 - accuracy: 0.9500 - val_loss: 0.4002 - val_accuracy: 0.8250 - 3s/epoch - 1s/step
Epoch 9/20
2/2 - 3s - loss: 0.2463 - accuracy: 0.9500 - val_loss